In [ ]:
import os
import random
from utils.hand_model_lite import HandModelMJCFLite
import numpy as np
import transforms3d
import torch
import trimesh
from utils.hand_model_type import translation_names, rot_names, handmodeltype_to_joint_names, HandModelType


In [ ]:
mesh_path = "../data/meshdata"
data_path = "../data/dataset"

joint_names = handmodeltype_to_joint_names[HandModelType.SHADOW_HAND]


In [ ]:
hand_model = HandModelMJCFLite(HandModelType.SHADOW_HAND)


In [ ]:
grasp_code_list = []
for code in os.listdir(data_path):
    grasp_code_list.append(code[:-4])


In [ ]:
grasp_code = random.choice(grasp_code_list)
grasp_data = np.load(
    os.path.join(data_path, grasp_code+".npy"), allow_pickle=True)
object_mesh_origin = trimesh.load(os.path.join(
    mesh_path, grasp_code, "coacd/decomposed.obj"))
print(grasp_code)


In [ ]:
index = random.randint(0, len(grasp_data) - 1)


qpos = grasp_data[index]['qpos']
rot = np.array(transforms3d.euler.euler2mat(
    *[qpos[name] for name in rot_names]))
rot = rot[:, :2].T.ravel().tolist()
hand_pose = torch.tensor([qpos[name] for name in translation_names] + rot + [qpos[name]
                         for name in joint_names], dtype=torch.float, device="cpu").unsqueeze(0)
hand_model.set_parameters(hand_pose)
hand_mesh = hand_model.get_trimesh_data(0)
object_mesh = object_mesh_origin.copy().apply_scale(grasp_data[index]["scale"])


In [ ]:
(hand_mesh+object_mesh).show()
